In [1]:
import os
import cv2
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision.datasets import DatasetFolder
from torchvision import transforms, utils
import numpy as np
import pickle
import matplotlib.pyplot as plt

from PIL import Image
from matplotlib import cm
import matplotlib.pyplot as plt
from torchsummary import summary

# Pollen-Detection-Network

### Network

In [2]:
class FCNN(nn.Module):
    """
    This is a class for fully convolutional neural networks.
    
    It is a subclass of the Module class from torch.nn.
    See the torch.nn documentation for more information.
    """
    
    def __init__(self):
        """
        The constructor for FCNN class. The internal states of the network are initialized. 
        """
        
        super(FCNN, self).__init__()
        self.conv0 = nn.Conv2d(in_channels=1, out_channels=30, kernel_size=2, stride=2)
        
        self.conv1 = nn.Conv2d(in_channels=30, out_channels=30, kernel_size=5, stride=2)
        
        self.conv2 = nn.Conv2d(in_channels=30, out_channels=60, kernel_size=3, stride=1)
        self.conv2_drop = nn.Dropout2d()
        
        self.conv3 = nn.Conv2d(in_channels=60, out_channels=60, kernel_size=3, stride=2)
        self.conv3_drop = nn.Dropout2d()
        
        self.conv4 = nn.Conv2d(in_channels=60, out_channels=120, kernel_size=3, stride=1)
        
        self.conv5 = nn.Conv2d(in_channels=120, out_channels=120, kernel_size=3, stride=1)
        
        self.conv6 = nn.Conv2d(in_channels=120, out_channels=1, kernel_size=1, stride=1)
        
        
    def forward(self, data):
        """
        Defines the computation performed at every call.
        
        Parameters:
            data (torch.Tensor): The input that is evaluated by the network. 
                The network expects the input to be of 4 dimensions.
            
        Returns:
            x (torch.Tensor): The output of the network after evaluating it on the given input.
        """
        
        x = F.relu(self.conv0(data))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2_drop(self.conv2(x)))
        x = F.relu(self.conv3_drop(self.conv3(x)))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = torch.sigmoid(self.conv6(x))
        return x

### Validation

In [30]:
def validate(network, testloader):
    """
    Calculates the mean validation loss, the F1-Score and the accuracy of the given network on the testloader data.
    
    Parameters:
        network (nn.Module): The network to validate. 
        testloader (torch.utils.data.DataLoader): Contains the data which is used to validate the network. 
        
    Returns:
        validation_loss (float): The mean loss per image with regards to a fixed criterion.
        F1_Score (float): The harmonic mean of the precision and the recall of the given network
            on the given validation data.
        correct / total (float): The percentage of correctly classified samples of the 
            total number of validation samples.
        
    """
    
    total = 0
    correct = 0
    true_positive = 0
    false_negative = 0
    false_positive = 0
    true_negative = 0

    # We do not need any gradiants here, since we do not train the network.
    # We are only interested in the predictions of the network on the testdata. 
    with torch.no_grad():
        validation_loss = 0
        #for i, sample in enumerate(testloader):
        for i, (inputs, labels) in enumerate(testloader):
            
 #           inputs = sample[0].to(device)
#            labels = sample[1].to(device)
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = network(torch.transpose(inputs[...,None],1,3)).view(-1)
            loss = criterion(outputs,labels)
            validation_loss += loss.item()
            predicted = (outputs >= 0.5) # Predicted is a tensor of booleans 
            total += labels.size(0)
            outputs = outputs.view(predicted.size(0))
            difference = (abs(outputs - labels))
            correct += (difference <= 0.1).sum().item()
            predicted = predicted.view(predicted.size(0)) 
            b_labels = labels != 0
            #correct += (predicted == b_labels).sum().item()
            true_positive += (predicted & b_labels).sum().item()
            false_negative += (np.logical_not(np.array(torch.Tensor.cpu(predicted))) & np.array(torch.Tensor.cpu(b_labels))).sum()
            
        f_validation_loss = validation_loss / total 
        true_negative = correct - true_positive
        false_positive = total - correct - false_negative
        
        print(true_positive)
        print(false_negative)
        print(false_positive)
        print(true_negative)
        
        try: 
            recall = true_positive / (true_positive + false_negative)
        except ZeroDivisionError:
            recall = 0
        try:
            precision = true_positive / (true_positive + false_positive)
        except ZeroDivisionError:
            precision = 0
        try:
            F1_score = 2 * (precision * recall) / (precision + recall)
        except ZeroDivisionError:
            F1_score = 0
            
    
    print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))
    print('F1 Score of the network on the test images: %f' % F1_score)
    return f_validation_loss, F1_score, (correct / total)

### Training

In [32]:
def train(network, trainloader, validloader, ep, criterion, optimizer, print_interval):
    """
    Train a neural network.
    
    
    Parameters:
        network (nn.Module): The network which should be trained.
        trainloader (torch.utils.data.DataLoader): Contains the data to be used to train the network.
        ep (int): The number of epochs the network will be trained for.
        criterion (): The loss function used for training (should be the same for validation).
        optimizer (): The specification how the weights are optimized.
        print_interval (int): The number of batches after which the function prints the loss for one batch.
    
    Returns:
        train_losses ([float]): The mean of the loss for one train sample after every epoch. 
        validation_losses ([float]): The mean of the loss for one validation sample after every epoch.
        F1 ([float]): The F1-Score of the validation data after every epoch.
        accuracy ([float]): The accuracy of the network on the validation data after every epoch.
    """
    
    losses = [] # Mean of loss/image in every epoch of training 
    validation_losses = []
    F1 = []
    accuracy = []
    print('Performance of the untrained network:')
    validate(network, validloader)
    
    for epoch in range(ep):

        loss_stats = 0.0
        total = 0
        
        running_loss = 0.0
    
        #for i, sample in enumerate(trainloader):
        for i, (inputs, labels) in enumerate(trainloader):    # this is some old code
            
            #inputs = sample[0].to(device)
            #labels = sample[1].to(device)
            inputs, labels = inputs.to(device), labels.to(device)    # this is some old code
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            outputs = network(torch.transpose(inputs[...,None],1,3)).view(-1)
            loss = criterion(outputs, labels)
            loss.backward() #propagate the error back through the network
            optimizer.step() #adjust the weights of the network depending on the propagated error
    
            #that's it.
            total += labels.shape[0]
            
            #Some statistics:
            running_loss += loss.item()
            
            loss_stats += loss.item()
            
            if i % print_interval == print_interval - 1:    # print every x mini-batches (loss for one batch)
                print('[Epoch: %d, Batch: %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / print_interval))
                running_loss = 0.0
        
        losses.append(loss_stats / total) 
        #Test the network using the validation set after every epoch of training.
        val_loss_curr, F1_curr, accuracy_curr = validate(network, validloader)
        validation_losses.append(val_loss_curr)
        F1.append(F1_curr)
        accuracy.append(accuracy_curr)
        
    print('Finished Training')
    return losses, validation_losses, F1, accuracy

### Create Dataset

In [33]:
class LabelStrengthDataset(DatasetFolder):
    """A generic data loader where the samples are arranged in this way: ::
        root/class_x/smpl1
        root/class_x/smpl2
        
        root/class_y/smpl3
        root/class_y/smpl4
        p
        with a single dictionary containing all samples as keys and their labelstrength as values
        
    Args:
        root (String):    path to root
        ls_file (String): path to pickle file containing the dictionary
    
    Attributes:
        root: path to root
        ls: dictionary with image names as keys and labelstrengths as values
        transform: transformations applied on samples
        samples: List of tuples containing a path to an image and its label strength
    """
    
    def __init__(self, root, ls_file, transform=None):
        with open(ls_file, 'rb') as pklfile:
            self.ls = pickle.load(pklfile)
        self.root = root
        self.transform = transform
        self.samples = self.make_dataset()
    
    def __len__(self):
        return len(self.ls)

    def __getitem__(self,index):
        img_path = self.samples[index][0]
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        np_image = np.array(image)
        t_image = torch.Tensor(np_image)
        labelstrength = self.samples[index][1]
        t_ls = torch.tensor(labelstrength).float()
        sample = (t_image, t_ls)
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
    
    def make_dataset(self):
        """Creates List of Tulpes containing path to img and its labelstrength"""
        print("Create Dataset..")
        instances = []
        for root, folders, files in os.walk(self.root):
            for file in files:
                if file.lower().endswith('.png'):
                    img_path = os.path.join(root,file)
                    label = self.ls[file]
#                     t_label = torch.tensor(label)
                    item = img_path, label
                    instances.append(item)
        print("Dataset created.")
        return instances

In [6]:
cwd = os.getcwd()
cwd

'/Users/mara/Desktop/SWP Über- und unbewachtes Lernen/pollen-detection'

In [7]:
root = os.path.join(cwd, 'Datasets/PollenData0710/')
pickle_dir = os.path.join(root, 'labelstrength.pickle')

In [8]:
dataset = LabelStrengthDataset(root, pickle_dir)

Create Dataset..
Dataset created.


In [25]:
dataset.__getitem__(1151)[1]

tensor(0.)

### Create Train- and Testsets

In [26]:
batch_size = 32
train_ratio = .8
shuffle_dataset = True
seed = np.random.randint
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(train_ratio * dataset_size))

if shuffle_dataset:
    np.random.shuffle(indices)

train_indices, valid_indices = indices[:split], indices[split:]
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)
torch.save(train_loader, 'trainloader.pth')
torch.save(valid_loader, 'validloader.pth')

# Train Network

#### Now we can train our network with given data

#### Initialise the network

In [34]:
# Setup wether gpu or cpu should be used for computations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create a new instance of the FCNN class. This will be our network.      
pollen_network = FCNN().to(device)

# Choose optimizer, criterion, number of epochs and threshold for the network to accept or dismiss a computated input
criterion = nn.MSELoss()
optimizer = optim.SGD(pollen_network.parameters(), lr=0.001, momentum=0.9)
ep = 10
threshold = 0.8

### Test untrained network

In [35]:
val_loss_curr, F1_curr, accuracy_curr = validate(pollen_network, valid_loader)

5119
32803
341275
2223
Accuracy of the network on the test images: 1 %
F1 Score of the network on the test images: 0.026640


### Train Network

In [36]:
train_losses, val_losses, F1_scores, accuracies = train(pollen_network, train_loader, valid_loader, ep, criterion, optimizer, 32)

Performance of the untrained network:
5165
32757
341289
2209
Accuracy of the network on the test images: 1 %
F1 Score of the network on the test images: 0.026875
[Epoch: 1, Batch:    32] loss: 0.075
[Epoch: 1, Batch:    64] loss: 0.057
[Epoch: 1, Batch:    96] loss: 0.053
[Epoch: 1, Batch:   128] loss: 0.056
[Epoch: 1, Batch:   160] loss: 0.065
[Epoch: 1, Batch:   192] loss: 0.069
[Epoch: 1, Batch:   224] loss: 0.069
[Epoch: 1, Batch:   256] loss: 0.057
[Epoch: 1, Batch:   288] loss: 0.063
[Epoch: 1, Batch:   320] loss: 0.059
[Epoch: 1, Batch:   352] loss: 0.055
[Epoch: 1, Batch:   384] loss: 0.060
[Epoch: 1, Batch:   416] loss: 0.061
[Epoch: 1, Batch:   448] loss: 0.055
[Epoch: 1, Batch:   480] loss: 0.062
[Epoch: 1, Batch:   512] loss: 0.054
[Epoch: 1, Batch:   544] loss: 0.054
[Epoch: 1, Batch:   576] loss: 0.060
[Epoch: 1, Batch:   608] loss: 0.053
[Epoch: 1, Batch:   640] loss: 0.057
[Epoch: 1, Batch:   672] loss: 0.062
[Epoch: 1, Batch:   704] loss: 0.050
[Epoch: 1, Batch:   736]

[Epoch: 1, Batch:  7008] loss: 0.051
[Epoch: 1, Batch:  7040] loss: 0.046
[Epoch: 1, Batch:  7072] loss: 0.041
[Epoch: 1, Batch:  7104] loss: 0.057
[Epoch: 1, Batch:  7136] loss: 0.053
[Epoch: 1, Batch:  7168] loss: 0.046
[Epoch: 1, Batch:  7200] loss: 0.048
[Epoch: 1, Batch:  7232] loss: 0.041
[Epoch: 1, Batch:  7264] loss: 0.046
[Epoch: 1, Batch:  7296] loss: 0.041
[Epoch: 1, Batch:  7328] loss: 0.052
[Epoch: 1, Batch:  7360] loss: 0.054
[Epoch: 1, Batch:  7392] loss: 0.046
[Epoch: 1, Batch:  7424] loss: 0.049
[Epoch: 1, Batch:  7456] loss: 0.044
[Epoch: 1, Batch:  7488] loss: 0.046
[Epoch: 1, Batch:  7520] loss: 0.054
[Epoch: 1, Batch:  7552] loss: 0.057
[Epoch: 1, Batch:  7584] loss: 0.040
[Epoch: 1, Batch:  7616] loss: 0.046
[Epoch: 1, Batch:  7648] loss: 0.045
[Epoch: 1, Batch:  7680] loss: 0.043
[Epoch: 1, Batch:  7712] loss: 0.043
[Epoch: 1, Batch:  7744] loss: 0.054
[Epoch: 1, Batch:  7776] loss: 0.054
[Epoch: 1, Batch:  7808] loss: 0.057
[Epoch: 1, Batch:  7840] loss: 0.051
[

[Epoch: 1, Batch: 14112] loss: 0.048
[Epoch: 1, Batch: 14144] loss: 0.035
[Epoch: 1, Batch: 14176] loss: 0.050
[Epoch: 1, Batch: 14208] loss: 0.042
[Epoch: 1, Batch: 14240] loss: 0.044
[Epoch: 1, Batch: 14272] loss: 0.040
[Epoch: 1, Batch: 14304] loss: 0.045
[Epoch: 1, Batch: 14336] loss: 0.043
[Epoch: 1, Batch: 14368] loss: 0.043
[Epoch: 1, Batch: 14400] loss: 0.040
[Epoch: 1, Batch: 14432] loss: 0.044
[Epoch: 1, Batch: 14464] loss: 0.039
[Epoch: 1, Batch: 14496] loss: 0.046
[Epoch: 1, Batch: 14528] loss: 0.048
[Epoch: 1, Batch: 14560] loss: 0.033
[Epoch: 1, Batch: 14592] loss: 0.045
[Epoch: 1, Batch: 14624] loss: 0.039
[Epoch: 1, Batch: 14656] loss: 0.040
[Epoch: 1, Batch: 14688] loss: 0.044
[Epoch: 1, Batch: 14720] loss: 0.049
[Epoch: 1, Batch: 14752] loss: 0.041
[Epoch: 1, Batch: 14784] loss: 0.043
[Epoch: 1, Batch: 14816] loss: 0.040
[Epoch: 1, Batch: 14848] loss: 0.043
[Epoch: 1, Batch: 14880] loss: 0.045
[Epoch: 1, Batch: 14912] loss: 0.039
[Epoch: 1, Batch: 14944] loss: 0.044
[

[Epoch: 1, Batch: 21216] loss: 0.034
[Epoch: 1, Batch: 21248] loss: 0.043
[Epoch: 1, Batch: 21280] loss: 0.027
[Epoch: 1, Batch: 21312] loss: 0.045
[Epoch: 1, Batch: 21344] loss: 0.036
[Epoch: 1, Batch: 21376] loss: 0.042
[Epoch: 1, Batch: 21408] loss: 0.028
[Epoch: 1, Batch: 21440] loss: 0.036
[Epoch: 1, Batch: 21472] loss: 0.033
[Epoch: 1, Batch: 21504] loss: 0.040
[Epoch: 1, Batch: 21536] loss: 0.031
[Epoch: 1, Batch: 21568] loss: 0.041
[Epoch: 1, Batch: 21600] loss: 0.035
[Epoch: 1, Batch: 21632] loss: 0.038
[Epoch: 1, Batch: 21664] loss: 0.041
[Epoch: 1, Batch: 21696] loss: 0.033
[Epoch: 1, Batch: 21728] loss: 0.039
[Epoch: 1, Batch: 21760] loss: 0.033
[Epoch: 1, Batch: 21792] loss: 0.037
[Epoch: 1, Batch: 21824] loss: 0.038
[Epoch: 1, Batch: 21856] loss: 0.040
[Epoch: 1, Batch: 21888] loss: 0.043
[Epoch: 1, Batch: 21920] loss: 0.047
[Epoch: 1, Batch: 21952] loss: 0.034
[Epoch: 1, Batch: 21984] loss: 0.037
[Epoch: 1, Batch: 22016] loss: 0.038
[Epoch: 1, Batch: 22048] loss: 0.035
[

[Epoch: 1, Batch: 28320] loss: 0.032
[Epoch: 1, Batch: 28352] loss: 0.039
[Epoch: 1, Batch: 28384] loss: 0.038
[Epoch: 1, Batch: 28416] loss: 0.032
[Epoch: 1, Batch: 28448] loss: 0.039
[Epoch: 1, Batch: 28480] loss: 0.032
[Epoch: 1, Batch: 28512] loss: 0.041
[Epoch: 1, Batch: 28544] loss: 0.042
[Epoch: 1, Batch: 28576] loss: 0.037
[Epoch: 1, Batch: 28608] loss: 0.039
[Epoch: 1, Batch: 28640] loss: 0.033
[Epoch: 1, Batch: 28672] loss: 0.031
[Epoch: 1, Batch: 28704] loss: 0.034
[Epoch: 1, Batch: 28736] loss: 0.036
[Epoch: 1, Batch: 28768] loss: 0.030
[Epoch: 1, Batch: 28800] loss: 0.035
[Epoch: 1, Batch: 28832] loss: 0.036
[Epoch: 1, Batch: 28864] loss: 0.034
[Epoch: 1, Batch: 28896] loss: 0.037
[Epoch: 1, Batch: 28928] loss: 0.035
[Epoch: 1, Batch: 28960] loss: 0.036
[Epoch: 1, Batch: 28992] loss: 0.037
[Epoch: 1, Batch: 29024] loss: 0.036
[Epoch: 1, Batch: 29056] loss: 0.035
[Epoch: 1, Batch: 29088] loss: 0.038
[Epoch: 1, Batch: 29120] loss: 0.035
[Epoch: 1, Batch: 29152] loss: 0.040
[

...

[Epoch: 10, Batch: 17024] loss: 0.020
[Epoch: 10, Batch: 17056] loss: 0.019
[Epoch: 10, Batch: 17088] loss: 0.012
[Epoch: 10, Batch: 17120] loss: 0.018
[Epoch: 10, Batch: 17152] loss: 0.012
[Epoch: 10, Batch: 17184] loss: 0.017
[Epoch: 10, Batch: 17216] loss: 0.014
[Epoch: 10, Batch: 17248] loss: 0.011
[Epoch: 10, Batch: 17280] loss: 0.011
[Epoch: 10, Batch: 17312] loss: 0.013
[Epoch: 10, Batch: 17344] loss: 0.012
[Epoch: 10, Batch: 17376] loss: 0.012
[Epoch: 10, Batch: 17408] loss: 0.013
[Epoch: 10, Batch: 17440] loss: 0.014
[Epoch: 10, Batch: 17472] loss: 0.016
[Epoch: 10, Batch: 17504] loss: 0.014
[Epoch: 10, Batch: 17536] loss: 0.011
[Epoch: 10, Batch: 17568] loss: 0.015
[Epoch: 10, Batch: 17600] loss: 0.017
[Epoch: 10, Batch: 17632] loss: 0.020
[Epoch: 10, Batch: 17664] loss: 0.013
[Epoch: 10, Batch: 17696] loss: 0.015
[Epoch: 10, Batch: 17728] loss: 0.013
[Epoch: 10, Batch: 17760] loss: 0.010
[Epoch: 10, Batch: 17792] loss: 0.012
[Epoch: 10, Batch: 17824] loss: 0.016
[Epoch: 10, 

[Epoch: 10, Batch: 23936] loss: 0.012
[Epoch: 10, Batch: 23968] loss: 0.016
[Epoch: 10, Batch: 24000] loss: 0.014
[Epoch: 10, Batch: 24032] loss: 0.012
[Epoch: 10, Batch: 24064] loss: 0.013
[Epoch: 10, Batch: 24096] loss: 0.013
[Epoch: 10, Batch: 24128] loss: 0.017
[Epoch: 10, Batch: 24160] loss: 0.013
[Epoch: 10, Batch: 24192] loss: 0.016
[Epoch: 10, Batch: 24224] loss: 0.012
[Epoch: 10, Batch: 24256] loss: 0.013
[Epoch: 10, Batch: 24288] loss: 0.013
[Epoch: 10, Batch: 24320] loss: 0.019
[Epoch: 10, Batch: 24352] loss: 0.014
[Epoch: 10, Batch: 24384] loss: 0.012
[Epoch: 10, Batch: 24416] loss: 0.014
[Epoch: 10, Batch: 24448] loss: 0.012
[Epoch: 10, Batch: 24480] loss: 0.011
[Epoch: 10, Batch: 24512] loss: 0.011
[Epoch: 10, Batch: 24544] loss: 0.018
[Epoch: 10, Batch: 24576] loss: 0.014
[Epoch: 10, Batch: 24608] loss: 0.014
[Epoch: 10, Batch: 24640] loss: 0.015
[Epoch: 10, Batch: 24672] loss: 0.012
[Epoch: 10, Batch: 24704] loss: 0.016
[Epoch: 10, Batch: 24736] loss: 0.012
[Epoch: 10, 

[Epoch: 10, Batch: 30848] loss: 0.017
[Epoch: 10, Batch: 30880] loss: 0.010
[Epoch: 10, Batch: 30912] loss: 0.014
[Epoch: 10, Batch: 30944] loss: 0.015
[Epoch: 10, Batch: 30976] loss: 0.014
[Epoch: 10, Batch: 31008] loss: 0.013
[Epoch: 10, Batch: 31040] loss: 0.011
[Epoch: 10, Batch: 31072] loss: 0.013
[Epoch: 10, Batch: 31104] loss: 0.017
[Epoch: 10, Batch: 31136] loss: 0.012
[Epoch: 10, Batch: 31168] loss: 0.017
[Epoch: 10, Batch: 31200] loss: 0.010
[Epoch: 10, Batch: 31232] loss: 0.012
[Epoch: 10, Batch: 31264] loss: 0.012
[Epoch: 10, Batch: 31296] loss: 0.012
[Epoch: 10, Batch: 31328] loss: 0.017
[Epoch: 10, Batch: 31360] loss: 0.014
[Epoch: 10, Batch: 31392] loss: 0.016
[Epoch: 10, Batch: 31424] loss: 0.014
[Epoch: 10, Batch: 31456] loss: 0.012
[Epoch: 10, Batch: 31488] loss: 0.013
[Epoch: 10, Batch: 31520] loss: 0.010
[Epoch: 10, Batch: 31552] loss: 0.013
[Epoch: 10, Batch: 31584] loss: 0.012
[Epoch: 10, Batch: 31616] loss: 0.010
[Epoch: 10, Batch: 31648] loss: 0.013
[Epoch: 10, 

[Epoch: 10, Batch: 37760] loss: 0.012
[Epoch: 10, Batch: 37792] loss: 0.013
[Epoch: 10, Batch: 37824] loss: 0.016
[Epoch: 10, Batch: 37856] loss: 0.012
[Epoch: 10, Batch: 37888] loss: 0.011
[Epoch: 10, Batch: 37920] loss: 0.010
[Epoch: 10, Batch: 37952] loss: 0.012
[Epoch: 10, Batch: 37984] loss: 0.012
[Epoch: 10, Batch: 38016] loss: 0.012
[Epoch: 10, Batch: 38048] loss: 0.009
[Epoch: 10, Batch: 38080] loss: 0.016
[Epoch: 10, Batch: 38112] loss: 0.014
[Epoch: 10, Batch: 38144] loss: 0.016
[Epoch: 10, Batch: 38176] loss: 0.011
[Epoch: 10, Batch: 38208] loss: 0.016
[Epoch: 10, Batch: 38240] loss: 0.012
[Epoch: 10, Batch: 38272] loss: 0.015
[Epoch: 10, Batch: 38304] loss: 0.016
[Epoch: 10, Batch: 38336] loss: 0.018
[Epoch: 10, Batch: 38368] loss: 0.013
[Epoch: 10, Batch: 38400] loss: 0.013
[Epoch: 10, Batch: 38432] loss: 0.013
[Epoch: 10, Batch: 38464] loss: 0.013
[Epoch: 10, Batch: 38496] loss: 0.017
[Epoch: 10, Batch: 38528] loss: 0.015
[Epoch: 10, Batch: 38560] loss: 0.012
[Epoch: 10, 

[Epoch: 10, Batch: 44672] loss: 0.015
[Epoch: 10, Batch: 44704] loss: 0.017
[Epoch: 10, Batch: 44736] loss: 0.013
[Epoch: 10, Batch: 44768] loss: 0.014
[Epoch: 10, Batch: 44800] loss: 0.013
[Epoch: 10, Batch: 44832] loss: 0.015
[Epoch: 10, Batch: 44864] loss: 0.010
[Epoch: 10, Batch: 44896] loss: 0.016
[Epoch: 10, Batch: 44928] loss: 0.014
[Epoch: 10, Batch: 44960] loss: 0.010
[Epoch: 10, Batch: 44992] loss: 0.017
[Epoch: 10, Batch: 45024] loss: 0.014
[Epoch: 10, Batch: 45056] loss: 0.013
[Epoch: 10, Batch: 45088] loss: 0.012
[Epoch: 10, Batch: 45120] loss: 0.010
[Epoch: 10, Batch: 45152] loss: 0.013
[Epoch: 10, Batch: 45184] loss: 0.013
[Epoch: 10, Batch: 45216] loss: 0.014
[Epoch: 10, Batch: 45248] loss: 0.012
[Epoch: 10, Batch: 45280] loss: 0.015
[Epoch: 10, Batch: 45312] loss: 0.012
[Epoch: 10, Batch: 45344] loss: 0.015
[Epoch: 10, Batch: 45376] loss: 0.010
[Epoch: 10, Batch: 45408] loss: 0.016
[Epoch: 10, Batch: 45440] loss: 0.013
[Epoch: 10, Batch: 45472] loss: 0.015
[Epoch: 10, 

#### Save trained Network

In [37]:
# save the networks parameters to have the possibility to reload them later:
cur_dir = os.getcwd()
save_path = cur_dir + '/pollennet_oct.pt'
torch.save(pollen_network.state_dict(), save_path)

# save the corresponding losses, F1-Scores and Accuracies using pickle:

with open('train_losses_oct.obj', 'wb') as train_losses_file:
    pickle.dump(train_losses, train_losses_file)

with open('val_losses_oct.obj', 'wb') as val_losses_file:
    pickle.dump(val_losses, val_losses_file)
    
with open('F1_oct.obj', 'wb') as F1_file:
    pickle.dump(F1_scores, F1_file)
    
with open('accuracies_oct.obj', 'wb') as accuracies_file:
    pickle.dump(accuracies, accuracies_file)

In [39]:
train_losses_2, val_losses_2, F1_scores_2, accuracies_2 = train(pollen_network, train_loader, valid_loader, ep, criterion, optimizer, 1000)

Performance of the untrained network:
23099
14823
20001
323497
Accuracy of the network on the test images: 90 %
F1 Score of the network on the test images: 0.570191
[Epoch: 1, Batch:  1000] loss: 0.014
[Epoch: 1, Batch:  2000] loss: 0.014
[Epoch: 1, Batch:  3000] loss: 0.013
[Epoch: 1, Batch:  4000] loss: 0.013
[Epoch: 1, Batch:  5000] loss: 0.014
[Epoch: 1, Batch:  6000] loss: 0.013
[Epoch: 1, Batch:  7000] loss: 0.013
[Epoch: 1, Batch:  8000] loss: 0.014
[Epoch: 1, Batch:  9000] loss: 0.014
[Epoch: 1, Batch: 10000] loss: 0.013
[Epoch: 1, Batch: 11000] loss: 0.013
[Epoch: 1, Batch: 12000] loss: 0.014
[Epoch: 1, Batch: 13000] loss: 0.013
[Epoch: 1, Batch: 14000] loss: 0.014
[Epoch: 1, Batch: 15000] loss: 0.013
[Epoch: 1, Batch: 16000] loss: 0.014
[Epoch: 1, Batch: 17000] loss: 0.013
[Epoch: 1, Batch: 18000] loss: 0.013
[Epoch: 1, Batch: 19000] loss: 0.013
[Epoch: 1, Batch: 20000] loss: 0.013
[Epoch: 1, Batch: 21000] loss: 0.014
[Epoch: 1, Batch: 22000] loss: 0.013
[Epoch: 1, Batch: 230

[Epoch: 5, Batch: 17000] loss: 0.012
[Epoch: 5, Batch: 18000] loss: 0.011
[Epoch: 5, Batch: 19000] loss: 0.012
[Epoch: 5, Batch: 20000] loss: 0.012
[Epoch: 5, Batch: 21000] loss: 0.012
[Epoch: 5, Batch: 22000] loss: 0.013
[Epoch: 5, Batch: 23000] loss: 0.012
[Epoch: 5, Batch: 24000] loss: 0.012
[Epoch: 5, Batch: 25000] loss: 0.012
[Epoch: 5, Batch: 26000] loss: 0.011
[Epoch: 5, Batch: 27000] loss: 0.012
[Epoch: 5, Batch: 28000] loss: 0.012
[Epoch: 5, Batch: 29000] loss: 0.012
[Epoch: 5, Batch: 30000] loss: 0.012
[Epoch: 5, Batch: 31000] loss: 0.012
[Epoch: 5, Batch: 32000] loss: 0.012
[Epoch: 5, Batch: 33000] loss: 0.012
[Epoch: 5, Batch: 34000] loss: 0.012
[Epoch: 5, Batch: 35000] loss: 0.012
[Epoch: 5, Batch: 36000] loss: 0.012
[Epoch: 5, Batch: 37000] loss: 0.013
[Epoch: 5, Batch: 38000] loss: 0.012
[Epoch: 5, Batch: 39000] loss: 0.012
[Epoch: 5, Batch: 40000] loss: 0.012
[Epoch: 5, Batch: 41000] loss: 0.012
[Epoch: 5, Batch: 42000] loss: 0.012
[Epoch: 5, Batch: 43000] loss: 0.011
[

[Epoch: 9, Batch: 37000] loss: 0.011
[Epoch: 9, Batch: 38000] loss: 0.011
[Epoch: 9, Batch: 39000] loss: 0.010
[Epoch: 9, Batch: 40000] loss: 0.011
[Epoch: 9, Batch: 41000] loss: 0.011
[Epoch: 9, Batch: 42000] loss: 0.011
[Epoch: 9, Batch: 43000] loss: 0.011
[Epoch: 9, Batch: 44000] loss: 0.010
[Epoch: 9, Batch: 45000] loss: 0.012
[Epoch: 9, Batch: 46000] loss: 0.011
[Epoch: 9, Batch: 47000] loss: 0.011
22357
15565
10528
332970
Accuracy of the network on the test images: 93 %
F1 Score of the network on the test images: 0.631491
[Epoch: 10, Batch:  1000] loss: 0.011
[Epoch: 10, Batch:  2000] loss: 0.011
[Epoch: 10, Batch:  3000] loss: 0.011
[Epoch: 10, Batch:  4000] loss: 0.011
[Epoch: 10, Batch:  5000] loss: 0.011
[Epoch: 10, Batch:  6000] loss: 0.011
[Epoch: 10, Batch:  7000] loss: 0.011
[Epoch: 10, Batch:  8000] loss: 0.010
[Epoch: 10, Batch:  9000] loss: 0.011
[Epoch: 10, Batch: 10000] loss: 0.011
[Epoch: 10, Batch: 11000] loss: 0.011
[Epoch: 10, Batch: 12000] loss: 0.011
[Epoch: 10

In [40]:
# save the networks parameters to have the possibility to reload them later:
cur_dir = os.getcwd()
save_path = cur_dir + '/pollennet_oct_2.pt'
torch.save(pollen_network.state_dict(), save_path)

# save the corresponding losses, F1-Scores and Accuracies using pickle:

with open('train_losses_oct_2.obj', 'wb') as train_losses_file:
    pickle.dump(train_losses, train_losses_file)

with open('val_losses_oct_2.obj', 'wb') as val_losses_file:
    pickle.dump(val_losses, val_losses_file)
    
with open('F1_oct_2.obj', 'wb') as F1_file:
    pickle.dump(F1_scores, F1_file)
    
with open('accuracies_oct_2.obj', 'wb') as accuracies_file:
    pickle.dump(accuracies, accuracies_file)

## Old Stuff

### Create Data

In [ ]:
# def load_pollen_data(dir):
#     np_pollen_data = []     # single date will be [img,label]
#     for folder in next(os.walk(dir))[1]:
#         if folder == 'p':
#             label = 1
#         else:
#             label = 0
#         parent_path = os.path.join(dir, folder)
#         for file in os.listdir(parent_path):
#             if '.png' in file:
#                 try:
#                     path = os.path.join(parent_path, file)
#                     img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
#                     np_pollen_data.append([np.array(img), label])
#                 except Exception as e:
#                     print(folder, file, str(e))

#     data = np.random.shuffle(np_pollen_data)
#     print("Data created.")
#     return np_pollen_data

### Create Train and Test Data

In [ ]:
# def create_test_val_sets(data, ratio=0.8):
#     """
#     Splits the data into a training and a validation data set.
    
#     Parameters:
#         data ([[obj, obj]]): A list of lists, which contain a sample and the correponding label.
#         ratio (float): The ratio between the size of the validation and the training data set.
        
#     Returns:
#         train_x ([obj]): The samples of the training data set.
#         train_y ([obj]): The corresponding labels of the training data set.
#         valid_x ([obj]): The samples of the validation data set.
#         valid_y ([obj]): The corresponding labels of the validation data set.
    
#     """
#     images = []
#     labels = []

#     for i in range(len(data)):
#         images.append(data[i][0])
#         labels.append(data[i][1])
    
#     train_x = images[:int(len(images)*ratio)]
#     train_y = labels[:int(len(images)*ratio)]
#     valid_x = images[int(len(images)*ratio):]
#     valid_y = labels[int(len(images)*ratio):]
    
#     print("Sets created")
#     return train_x, train_y, valid_x, valid_y

#### Load Data

In [ ]:
# # Load Data
# cur_dir = os.getcwd()
# pollen_path = os.path.join(cur_dir, 'Datasets/PollenData64')
# pollendata = load_pollen_data(pollen_path)

In [ ]:
# # Split into sets
# train_x, train_y, valid_x, valid_y = create_test_val_sets(pollendata, 0.9)

# pollendata = []

# # Our neural network expects a dataloader
# trainloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.Tensor(train_x),torch.Tensor(train_y)),batch_size=32)
# validloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.Tensor(valid_x),torch.Tensor(valid_y)),batch_size=1)

# # Save dataloader for later use
# #torch.save(trainloader, 'trainloader.pth')
# #torch.save(validloader, 'validloader.pth')

#### Initialise the network

In [ ]:
# # Setup wether gpu or cpu should be used for computations
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Create a new instance of the FCNN class. This will be our network.      
# pollen_network = FCNN().to(device)

# # Choose optimizer, criterion, number of epochs and threshold for the network to accept or dismiss a computated input
# criterion = nn.MSELoss()
# optimizer = optim.SGD(pollen_network.parameters(), lr=0.001, momentum=0.9)
# ep = 10
# threshold = 0.8